In [1]:
from keras.applications.mobilenet import preprocess_input
from keras.models import load_model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import shap
import numpy as np
import tensorflow as tf

from PIL import Image







IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def read_image(image1):
    # image2 = tf.io.decode_jpeg(image1, channels=3)
    image2 = tf.image.convert_image_dtype(image1, tf.float32)
    image3 = tf.image.resize_with_pad(image2, target_height=256, target_width=256)
    return image3

custom_objects = {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m}


In [3]:
from keras.models import load_model
model = load_model('./SavedModelPlantDetectionMLModel3.h5',custom_objects=custom_objects)

In [4]:
import os
from sklearn.model_selection import train_test_split

# Path to your local directory containing image data
data_dir = './valid/valid/'

# Get the list of all image files in the directory
scab_images = [os.path.join(data_dir, 'Apple___Apple_scab', img) for img in os.listdir(os.path.join(data_dir, 'Apple___Apple_scab'))]
rot_images = [os.path.join(data_dir, 'Apple___Black_rot', img) for img in os.listdir(os.path.join(data_dir, 'Apple___Black_rot'))]
rust_images = [os.path.join(data_dir, 'Apple___Cedar_apple_rust', img) for img in os.listdir(os.path.join(data_dir, 'Apple___Cedar_apple_rust'))]
healthy_images = [os.path.join(data_dir, 'Apple___healthy', img) for img in os.listdir(os.path.join(data_dir, 'Apple___healthy'))]

# Combine images into a single list
all_images = scab_images + rot_images + rust_images + healthy_images

# Create labels for different classes
scab_labels = [0] * len(scab_images)
rot_labels = [1] * len(rot_images)
rust_labels = [2] * len(rust_images)
healthy_labels = [3] * len(healthy_images)

# Combine labels into a single list
all_labels = scab_labels + rot_labels + rust_labels + healthy_labels

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)


In [5]:
len(xtrain)

1554

In [6]:
xtrain[0]

'./valid/valid/Apple___healthy\\77b48eb5-7237-460f-8751-cfb231c917a7___RS_HL 7687.JPG'

In [7]:
import cv2
import numpy as np

def convert_images_to_arrays(image_paths):
    image_arrays = []

    for image_path in image_paths:
        # Read the image using OpenCV
        image = cv2.imread(image_path)

        # Convert the image to RGB format (OpenCV reads images in BGR format)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Convert the image to a NumPy array
        image_array = np.array(image_rgb)

        # Append the NumPy array to the list
        image_arrays.append(image_array)
    
    image_arrays1 = np.array(image_arrays)
    return image_arrays1

In [8]:
xtrain_arrays = convert_images_to_arrays(xtrain)

In [9]:
len(xtrain_arrays)

1554

In [10]:
type(xtrain_arrays)

numpy.ndarray

In [11]:
xtrain_arrays.shape


(1554, 256, 256, 3)

In [12]:
xtrain_arrays[0]

array([[[162, 163, 194],
        [164, 165, 196],
        [166, 167, 198],
        ...,
        [160, 169, 200],
        [160, 169, 200],
        [160, 169, 200]],

       [[162, 163, 194],
        [164, 165, 196],
        [166, 167, 198],
        ...,
        [160, 169, 200],
        [160, 169, 200],
        [161, 170, 201]],

       [[162, 163, 194],
        [164, 165, 196],
        [165, 166, 197],
        ...,
        [160, 169, 200],
        [161, 170, 201],
        [161, 170, 201]],

       ...,

       [[ 98,  93, 123],
        [ 94,  89, 119],
        [ 91,  86, 116],
        ...,
        [118, 119, 150],
        [120, 121, 152],
        [140, 141, 172]],

       [[ 92,  87, 117],
        [ 92,  87, 117],
        [ 93,  88, 118],
        ...,
        [126, 127, 158],
        [134, 135, 166],
        [108, 109, 140]],

       [[ 86,  81, 111],
        [ 91,  86, 116],
        [ 97,  92, 122],
        ...,
        [114, 115, 146],
        [127, 128, 159],
        [119, 120, 151]]

In [19]:
xtrain_arrays_float32 = xtrain_arrays.astype('float32') / 255.0

In [21]:
e = shap.DeepExplainer(model, xtrain_arrays_float32)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.